# Shape Classifier: Circles, Squares, and Triangles

A Jupyter Notebook to demonstrate the training and evaluation of a CNN model to classify synthetic images of geometric shapes.

**Project Structure Note:** In adherence to the project requirement for `clean, modular, and well-organized code,` complex helper functions for data generation and visualization are encapsulated in the `ShapeClassifier/utils/` directory. This keeps the main notebook focused on the core machine learning workflow: configuration, data loading, model definition, training, and evaluation.

## 1. Setup & Configuration

This section handles all the initial setup for the project:
*   Importing essential libraries.
*   Defining constants, hyperparameters, and file paths.
*   Setting up reproducibility seeds.

### 1.1. Path Configuration

First, we set up the system path to ensure our utility scripts in the `ShapeClassifier/utils/` directory can be imported reliably. We also define the core paths for our project artifacts.

In [2]:
import os
import sys

# We assume this notebook is run from its location in the 'ShapeClassifier' directory.
# The project root is one level up.
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"Added '{PROJECT_ROOT}' to system path.")

# Define core paths relative to the project root for robustness.
ARTIFACTS_DIR = os.path.join(PROJECT_ROOT, '..', 'shape-classifier-artifacts')
DATASET_DIR = os.path.join(ARTIFACTS_DIR, 'shape-classifier-datasets', 'ShapeClassifier')
MODEL_CHECKPOINT_DIR = os.path.join(ARTIFACTS_DIR, 'shape-classifier-models', 'ShapeClassifier')

print(f"Project Root: {PROJECT_ROOT}")
print(f"Dataset Directory: {DATASET_DIR}")

Added 'c:\Users\gilda\OneDrive\Documents\My_Projects\AI_Based_Projects\shape-classifier-interview\shape-classifier-pytorch' to system path.
Project Root: c:\Users\gilda\OneDrive\Documents\My_Projects\AI_Based_Projects\shape-classifier-interview\shape-classifier-pytorch
Dataset Directory: c:\Users\gilda\OneDrive\Documents\My_Projects\AI_Based_Projects\shape-classifier-interview\shape-classifier-pytorch\..\shape-classifier-artifacts\shape-classifier-datasets\ShapeClassifier


### 1.2. Library Imports

Next, we import all the necessary libraries for the project.

In [8]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

print("Libraries imported successfully.")

Libraries imported successfully.


### 1.3. Constants and Seeding

We define key constants for our project and a function to set seeds for reproducibility. The seeding function is called in a separate cell to prevent it from being re-run unnecessarily when constants are changed.

In [9]:
# --- Constants ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CLASSES = ['circle', 'square', 'triangle']
NUM_CLASSES = len(CLASSES)
MODEL_CHECKPOINT_PATH = os.path.join(MODEL_CHECKPOINT_DIR, 'best_model_checkpoint.pth')


# --- Seeding Function ---
def set_seed(seed=42):
    """Sets the seed for random, numpy, and torch for reproducible results."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    print(f"Seed set to {seed}")

print(f"Device set to: {DEVICE.upper()}")

Device set to: CUDA


### 1.4. Initial Setup Execution

Here we execute one-time setup commands: we set the global seed and ensure the directory for saving our model checkpoints exists.

In [10]:
# Set the seed for the entire notebook session.
set_seed(42)

# Ensure the directory for saving model checkpoints exists.
os.makedirs(MODEL_CHECKPOINT_DIR, exist_ok=True)
print("Initial setup executed.")

Seed set to 42
Initial setup executed.


## 2. Data Generation & Loading

This section covers preparing the data for the model. It includes:
*   An optional step to regenerate the entire dataset from scratch.
*   Defining a custom PyTorch `Dataset` class to load the images.
*   Applying necessary transformations.
*   Creating `DataLoader`s to feed data to the model in batches.

### 2.1. Optional: Generate Dataset from Scratch

This project is configured to use the pre-generated (or downloaded) dataset located in the `shape-classifier-artifacts` directory.

However, if you wish to regenerate the entire dataset from scratch, you can set the `REGENERATE_DATA` flag in the cell below to `True`.

**Warning:** This process will delete all existing data in the target directory and will take several minutes to complete.

In [12]:
from ShapeClassifier.utils.data_generator import generate_dataset

# --- Configuration for Data Generation ---
REGENERATE_DATA = False # Set to True to run the data generation process.

# These parameters are used ONLY if REGENERATE_DATA is True.
TOTAL_IMAGES_TO_GENERATE = 6000
SPLIT_RATIOS = {'train': 0.7, 'validation': 0.15, 'test': 0.15}


if REGENERATE_DATA:
    # We call our imported function using the path constants defined in Section 1.
    generate_dataset(
        root_dir=DATASET_DIR,
        total_images=TOTAL_IMAGES_TO_GENERATE,
        # root_dir=os.path.join(DATASET_DIR, "Delete"),
        # total_images=100,
        splits=SPLIT_RATIOS,
        shapes=CLASSES # Using the 'CLASSES' constant from Section 1
    )
else:
    print("Skipping data generation. Using existing dataset.")

# Verify that the dataset directory exists.
if not os.path.exists(DATASET_DIR):
    print("\nERROR: Dataset directory not found!")
    print(f"Please run the project's download script or set REGENERATE_DATA to True.")
else:
    print(f"\nDataset found at: {DATASET_DIR}")

Skipping data generation. Using existing dataset.

Dataset found at: c:\Users\gilda\OneDrive\Documents\My_Projects\AI_Based_Projects\shape-classifier-interview\shape-classifier-pytorch\..\shape-classifier-artifacts\shape-classifier-datasets\ShapeClassifier


## 3. Model Architecture

Here, we define the architecture of our Convolutional Neural Network (CNN) and provide the justification for this choice.

## 4. Training the Model

This section contains the logic for training the model, including:
*   Defining the loss function and optimizer.
*   Implementing the training and validation loops.
*   Incorporating early stopping and model checkpointing to save the best-performing model.

## 5. Evaluation

After training, we evaluate the model's performance on the unseen test set. This includes:
*   Loading the best model checkpoint.
*   Calculating final test accuracy.
*   Visualizing a confusion matrix and sample predictions.

## 6. Analysis & Conclusion

A final summary of the results, including:
*   Plotting the training/validation curves.
*   Answering the project questions on learnings and potential improvements.